In [ ]:
# !pip install --upgrade pip
# !pip install gensim
# !pip install nltk
# !pip install tokenizers
# !pip install sentencepiece
# !pip install python-bidi
# !pip install arabic-reshaper
# !pip install PyArabic
# !pip install spacy

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
# from WordFeatureExtraction import WordFeatureExtraction
import pandas as pd
from pyarabic.araby import tokenize, strip_tashkeel
from tqdm import tqdm

from sklearn.metrics import accuracy_score
import numpy as np
from utilities import *
import textProcessing as tp
import nltk
from Embedding_Dataset import Embedding_Dataset
# nltk.download('punkt')
# train_text = load_text("dataset/train.txt")
# tp.preprocessing_text(train_text,"train_preprocessed.txt")


DIACRITIC2INDEX:  dict_items([('ً', 0), ('ٌ', 1), ('ٍ', 2), ('َ', 3), ('ُ', 4), ('ِ', 5), ('ّ', 6), ('ًّ', 7), ('ٌّ', 8), ('ٍّ', 9), ('َّ', 10), ('ُّ', 11), ('ِّ', 12), ('ْ', 13), ('', 14), ('0', 15)])
{'ً': 0, 'ٌ': 1, 'ٍ': 2, 'َ': 3, 'ُ': 4, 'ِ': 5, 'ّ': 6, 'ًّ': 7, 'ٌّ': 8, 'ٍّ': 9, 'َّ': 10, 'ُّ': 11, 'ِّ': 12, 'ْ': 13, '': 14, '0': 15}
16
{'ء': 0, 'آ': 1, 'أ': 2, 'ؤ': 3, 'إ': 4, 'ئ': 5, 'ا': 6, 'ب': 7, 'ة': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ك': 28, 'ل': 29, 'م': 30, 'ن': 31, 'ه': 32, 'و': 33, 'ى': 34, 'ي': 35, ' ': 36, '0': 37, 's': 38}
39


In [ ]:
# Download via terminal commands
# !wget "https://archive.org/download/aravec2.0/tweet_cbow_300.zip"
# !unzip "tweet_cbow_300.zip"

In [21]:
# import gensim
# from gensim.models import KeyedVectors
# from gensim.models import word2vec
# w2v_embeddings_index={}
# TOTAL_EMBEDDING_DIM=300
# w2v_model =KeyedVectors.load("tweets_cbow_300")
# for word in w2v_model.wv.index_to_key:
#     w2v_embeddings_index[word] = w2v_model.wv.get_vector(word)
# print(len(w2v_embeddings_index))

331679


In [ ]:
# # create a weight matrix for words in training docs
# embedding_matrix = np.zeros((vocab_size, TOTAL_EMBEDDING_DIM))
# for word, i in t.word_index.items():
#     embedding_vector = w2v_embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector
# print(“Embedding Matrix shape:”, embedding_matrix.shape)

In [2]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNClassifier, self).__init__()

        self.hidden_size = hidden_size
        self.input_size = input_size
        # self.wordEmbedding=WordFeatureExtraction()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        # self.out = nn.Sequential(nn.Linear(hidden_size, 64),nn.Tanh(),nn.Linear(64, output_size),nn.Softmax(dim=1))
        self.linear1 = nn.Linear(hidden_size, 64)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(64, output_size)
        # self.soft = nn.Softmax(dim=1)
        # self.out = nn.Sequential(nn.Linear(hidden_size, output_size),nn.Softmax())
        self.init_weight()

    def forward(self, input, hidden):

        output, hidden = self.rnn(input, hidden)
        # reshape the output to be able to pass it to the linear layer
        # output = output.contiguous().view(-1, self.hidden_size)
        output = self.linear1(output)
        output = self.tanh(output)
        output = self.linear2(output)
        # output = self.soft(output)
        return output

    def init_hidden(self, batch_size):
        return torch.zeros(1,batch_size, self.hidden_size)
    def init_weight(self):
        for name, param in self.named_parameters():
            if 'bias' in name:
                nn.init.constant(param, 0.0)
            elif 'weight' in name:
                nn.init.xavier_normal(param)

In [3]:
# Hyperparameter
input_size = 38
hidden_size = 64
output_size = 15
batch_size = 1024*8
num_epochs = 50

In [ ]:
# connect to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

# Create an instance of the RNN classifier
model = RNNClassifier(input_size, hidden_size, output_size)
model.to(device)
#######################################################################################
Embedding_Data=Embedding_Dataset()
train_dataloader = DataLoader(Embedding_Data, batch_size=1024*8, shuffle=True, collate_fn=Embedding_Data.collate_fn)
wordEmbeddingVectors ,labels_batches = Embedding_Data.extract_sentences_word_embedding(train_dataloader)
#########################################################################################
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

cpu
False
0


<ipython-input-2-d5a475f5a635>:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(param)
<ipython-input-2-d5a475f5a635>:33: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(param, 0.0)


0


In [ ]:
# Training loop
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_dataloader:
      # Reshape input and labels to (batch_size, seq_length, input_size)
      input = inputs.view(inputs.shape[0], -1, input_size)
      labels = labels.view(inputs.shape[0], -1, output_size)
      input, labels = input.to(device), labels.to(device)
      hidden = model.init_hidden(batch_size=inputs.shape[0])
      hidden = hidden.to(device)

      # Forward pass
      output = model(input, hidden)
      # Compute loss
      loss = criterion(output, labels)
      # Zero the gradients
      optimizer.zero_grad()
      # Backward pass and optimization
      loss.backward()
      optimizer.step()
      # Print loss for monitoring
      print(f"Epoch: {epoch+1}, Batch Loss: {loss.item()}")

